In [1]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from PIL import Image


In [2]:
df = pd.read_csv('/Users/user/repos/zoobot-3d/data/gz3d_and_gz_desi_matches.csv')

In [3]:
# df = df[:100]  # debugging mode

In [4]:
#  '/share/nas2/walml/galaxy_zoo/decals/dr8/fits/'
df['galahad_fits_loc'] = df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.fits'


#  '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/'
df['galahad_jpg_loc'] = df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'

df['galahad_fits_loc'][0], df['galahad_jpg_loc'][0]

('593252/593252_2670.fits', '593252/593252_2670.jpg')

In [16]:
# df['dr8_id'].value_counts()
# TODO drop duplicate matches, temporarily
df = df.drop_duplicates(subset='dr8_id', keep='False')

389903_2553    2
517154_288     2
571816_559     2
429117_822     2
490074_127     2
              ..
482509_1851    1
427433_3340    1
619690_2699    1
343992_4748    1
482215_716     1
Name: dr8_id, Length: 28994, dtype: int64

In [5]:
with open('data/galahad_fits_to_copy.txt', 'w') as f:
    f.write('\n'.join(df['galahad_fits_loc']))

with open('data/galahad_jpg_to_copy.txt', 'w') as f:
    f.write('\n'.join(df['galahad_jpg_loc']))

In [6]:
# rsync --files-from data/galahad_fits_to_copy.txt -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/fits data/gz_desi/fits

In [7]:
# rsync --files-from data/galahad_jpg_to_copy.txt -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/jpg data/gz_desi/jpg

In [8]:
df['local_desi_fits_loc'] = 'data/desi/fits/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.fits'
df['local_desi_jpg_loc'] = 'data/desi/jpg/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'

In [9]:
df['local_json_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/marks/').replace('.fits.gz', '.json'))

In [10]:
df['local_spiral_mask_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/masks/').replace('.fits.gz', '_spiral.jpg'))
df['local_bar_mask_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/masks/').replace('.fits.gz', '_bar.jpg'))

In [11]:
df['local_spiral_mask_loc'][0]

'data/gz3d/segmaps/masks/gz3d_1-379196_127_14728482_spiral.jpg'

In [12]:
len(df)

29005

In [13]:
df = df[df['local_gz3d_fits_loc'].apply(lambda x: os.path.isfile(x))]
len(df)

29005

In [14]:
# add GZ DESI votes

columns = [
    'dr8_id',
    'smooth-or-featured_featured-or-disk_fraction',
    'disk-edge-on_yes_fraction',
    'has-spiral-arms_yes_fraction',
    'spiral-arm-count_2_fraction'
]
gz_desi = pd.read_parquet(
    '/Users/user/repos/desi-predictions/results/final/gz_desi_deep_learning_catalog_advanced.parquet',
    columns=columns
)

# apply filtering, optionally
# gz_desi = gz_desi[gz_desi['smooth-or-featured_featured-or-disk_fraction'] > 0.5]
# gz_desi = gz_desi[gz_desi['has-spiral-arms_yes_fraction'] > 0.5]
# gz_desi = gz_desi[gz_desi['spiral-arm-count_2_fraction'] > 0.3]

df = pd.merge(df, gz_desi, on='dr8_id', validate='one_to_one', how='inner')

MergeError: Merge keys are not unique in left dataset; not a one-to-one merge

In [ ]:
df.to_csv('data/gz3d_and_gz_desi_master_catalog.csv', index=False)